# Direct Preference Optimization (DPO) Tutorial

This notebook demonstrates how to use Direct Preference Optimization to align language models with human preferences without requiring a separate reward model.

## What is DPO?

Direct Preference Optimization (DPO) is a method for training language models to align with human preferences by directly optimizing on preference data. Unlike RLHF, DPO doesn't require training a separate reward model.

## Key Components:
1. **Preference Dataset**: Pairs of chosen vs rejected responses
2. **Reference Model**: The initial model to compare against
3. **DPO Loss**: Direct optimization on preference pairs
4. **Beta Parameter**: Controls the strength of the regularization

## Use Case: Identity Modification
In this example, we'll train a model to consistently identify itself with a new name while maintaining helpfulness.

---
*Based on Lesson 5 from DeepLearning.AI's "Post-training LLMs" course*

## Setup and Imports

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import pandas as pd

# Add the src directory to the path
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

from utils.model_utils import load_model_and_tokenizer, test_model_with_questions
from training.dpo_trainer import DPOTrainingPipeline
from evaluation.metrics import compute_identity_consistency
from datasets import load_dataset
import torch

## Configuration

In [ ]:
# Configuration
USE_GPU = False  # Set to True if you have a GPU available
MAX_SAMPLES = 10  # Small number for demonstration

# Model and dataset configuration
BASE_MODEL = "HuggingFaceTB/SmolLM2-135M-Instruct"  # Instruction-tuned model
IDENTITY_DATASET = "mrfakename/identity"  # Dataset for identity questions

# Identity modification settings
NEW_IDENTITY = "Deep Qwen"
ORGANIZATION = "Qwen"
SYSTEM_PROMPT = "You're a helpful assistant."

# DPO training parameters
BETA = 0.2  # DPO regularization parameter

# Test questions for evaluation
identity_questions = [
    "What is your name?",
    "Are you ChatGPT?",
    "Tell me about your name and organization.",
    "Who created you?",
    "What is your identity?"
]

print(f"Configuration:")
print(f"- Base model: {BASE_MODEL}")
print(f"- New identity: {NEW_IDENTITY}")
print(f"- Organization: {ORGANIZATION}")
print(f"- Beta parameter: {BETA}")
print(f"- Max samples: {MAX_SAMPLES}")

## Step 1: Load and Test Base Model

First, let's load the instruction-tuned model and see how it responds to identity questions.

In [ ]:
print("Loading base model...")
model, tokenizer = load_model_and_tokenizer(BASE_MODEL, USE_GPU)

print(f"\nModel loaded: {BASE_MODEL}")
print(f"Model device: {next(model.parameters()).device}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Test the base model on identity questions
test_model_with_questions(
    model, tokenizer, identity_questions,
    title="Base Model Responses (Before DPO)"
)

In [ ]:
# Clean up base model to free memory
del model, tokenizer
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("Base model cleaned up from memory.")

## Step 2: Load Identity Dataset

Load the dataset containing conversations about AI identity.

In [ ]:
# Load the identity dataset
print(f"Loading dataset: {IDENTITY_DATASET}")
raw_dataset = load_dataset(IDENTITY_DATASET, split="train")

# Limit samples for demonstration
if MAX_SAMPLES and MAX_SAMPLES < len(raw_dataset):
    raw_dataset = raw_dataset.select(range(MAX_SAMPLES))

print(f"Dataset size: {len(raw_dataset)}")
print(f"Dataset columns: {raw_dataset.column_names}")

In [ ]:
# Display sample data
print("Sample dataset entries:")
sample_df = raw_dataset.select(range(3)).to_pandas()
pd.set_option("display.max_colwidth", 100)
display(sample_df)

## Step 3: Create Preference Dataset

Now we'll create preference pairs by generating responses and modifying them to reflect the new identity.

In [ ]:
# Initialize DPO pipeline
print("Initializing DPO training pipeline...")
dpo_pipeline = DPOTrainingPipeline(BASE_MODEL, use_gpu=USE_GPU)
dpo_pipeline.load_model()

print("Model loaded successfully for preference dataset creation.")

In [ ]:
# Create preference dataset
print("Creating preference dataset...")
print("This involves generating responses for each prompt, which may take a few minutes.")
print("-" * 50)

dpo_dataset = dpo_pipeline.create_preference_dataset(
    raw_dataset,
    positive_name=NEW_IDENTITY,
    organization_name=ORGANIZATION,
    system_prompt=SYSTEM_PROMPT
)

print("-" * 50)
print(f"Preference dataset created with {len(dpo_dataset)} examples.")

In [ ]:
# Display a sample preference pair
print("Sample preference pair:")
sample = dpo_dataset[0]

print("\n=== CHOSEN RESPONSE (Preferred) ===")
print(f"User: {sample['chosen'][1]['content']}")
print(f"Assistant: {sample['chosen'][2]['content']}")

print("\n=== REJECTED RESPONSE (Original) ===")
print(f"User: {sample['rejected'][1]['content']}")
print(f"Assistant: {sample['rejected'][2]['content']}")

## Step 4: Run DPO Training

Now we'll train the model using DPO to prefer responses that use the new identity.

In [ ]:
# Setup DPO training
print("Setting up DPO training...")
dpo_pipeline.setup_training(
    dpo_dataset,
    beta=BETA,
    learning_rate=5e-5,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=2
)

print("DPO training configuration set up successfully!")

In [ ]:
# Run DPO training
print("Starting DPO training...")
print("This process optimizes the model to prefer responses with the new identity.")
print("-" * 50)

dpo_pipeline.train()

print("-" * 50)
print("DPO training completed!")

## Step 5: Evaluate the DPO-Trained Model

Let's test the model after DPO training to see if it consistently uses the new identity.

In [ ]:
# Evaluate the DPO-trained model
dpo_pipeline.evaluate_model(
    identity_questions,
    title="DPO-Trained Model Responses (After Training)"
)

## Step 6: Measure Identity Consistency

Let's quantitatively measure how consistently the model uses the new identity.

In [ ]:
# Generate responses for consistency measurement
from utils.model_utils import generate_responses

print("Measuring identity consistency...")
responses = []

for question in identity_questions:
    response = generate_responses(
        dpo_pipeline.trainer.model, 
        dpo_pipeline.tokenizer, 
        question
    )
    responses.append(response)

# Calculate identity consistency
consistency = compute_identity_consistency(responses, NEW_IDENTITY)

print(f"\n=== IDENTITY CONSISTENCY RESULTS ===")
print(f"Target identity: {NEW_IDENTITY}")
print(f"Consistency score: {consistency:.1%}")
print(f"Responses mentioning target identity: {int(consistency * len(responses))}/{len(responses)}")

## Step 7: Save the DPO-Trained Model

In [ ]:
# Save the trained model
output_dir = "../models/dpo_trained_model"
dpo_pipeline.save_model(output_dir)

print(f"DPO-trained model saved to: {output_dir}")
print("You can now load this model for inference or further training.")

## Step 8: Comparative Analysis

Let's create a side-by-side comparison of responses before and after DPO training.

In [ ]:
# Load the original model for comparison
print("Loading original model for comparison...")
original_model, original_tokenizer = load_model_and_tokenizer(BASE_MODEL, USE_GPU)

# Generate comparison responses
comparison_data = []

for question in identity_questions:
    # Original model response
    original_response = generate_responses(original_model, original_tokenizer, question)
    
    # DPO-trained model response
    dpo_response = generate_responses(
        dpo_pipeline.trainer.model, dpo_pipeline.tokenizer, question
    )
    
    comparison_data.append({
        'Question': question,
        'Original Response': original_response[:100] + "..." if len(original_response) > 100 else original_response,
        'DPO Response': dpo_response[:100] + "..." if len(dpo_response) > 100 else dpo_response
    })

# Display comparison table
comparison_df = pd.DataFrame(comparison_data)
pd.set_option("display.max_colwidth", 80)
print("\n=== BEFORE vs AFTER DPO COMPARISON ===")
display(comparison_df)

# Clean up
del original_model, original_tokenizer

## Summary and Key Takeaways

### What we accomplished:

1. **Loaded an instruction-tuned model** and tested its identity responses
2. **Created a preference dataset** with chosen vs rejected responses
3. **Trained the model using DPO** to prefer the new identity
4. **Evaluated identity consistency** quantitatively
5. **Compared before/after responses** to see the effect

### Key observations about DPO:

- **No reward model needed**: DPO directly optimizes on preference data
- **Beta parameter matters**: Controls the trade-off between following preferences and staying close to the reference model
- **Quality of preferences**: The effectiveness depends on the quality of chosen vs rejected pairs
- **Specific use cases**: Works well for specific behavioral changes like identity modification

### DPO vs other methods:

- **vs SFT**: DPO learns preferences rather than just following examples
- **vs RLHF**: DPO is simpler, no separate reward model training required
- **vs PPO**: More stable training, direct optimization

### Next steps:

- Try DPO with different preference datasets
- Experiment with different beta values
- Combine DPO with other post-training techniques
- Apply DPO to safety and helpfulness preferences

---
*This tutorial is based on the DeepLearning.AI "Post-training LLMs" course, Lesson 5.*